<h1 style='font-size:40px'> Extending LangChain</h1>

<h2 style='font-size:30px'> Testing the Streaming Setup</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>   
            Uma vez implantada a nossa chain com suporte a streaming, repararemos num erro do Flask que afirma que a Thread criada não tem acesso ao cotexto de nossa aplicação.
            <center style='margin-top:20px'> 
                <img src='img/11_thread_context.png'>
            </center>
        </li>
        <li style='margin-top:20px'>
            O contexto do site contém metadados importantes para a sua execução.
        </li>
        <li>
            Felizmente, o Flask tem uma variável (a `current_app`), que possibilita que nós invoquemos o contexto da aplicação dentro do contexto de um objeto particular.
        </li>
    </ul>
 </div>

<figure>
    <center style='margin-top:20px'>
        <img src='img/11_current_app.png'>
            <figcaption> Implementação do `current_app` na classe de nossa `StreamableChain`.</figcaption>
    </center>
</figure>

<!-- <h3 style='font-size:30px;font-style:italic'> </h3> -->
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Uma vez implantada essa correção, repararemos um outro comportamento pouco esperado de nossa aplicação: o modelo parece que não consegue entender nossas questões!
            <center style='margin-top:20px'> 
                <img src='img/11_strange_behaviour.png'>
            </center>
        </li>
    </ul>
 </div>

<h2 style='font-size:30px'> Here's the Issue</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>   
            Devemos nos lembrar que a chain do projeto é, na verdade, um wrapper de outras duas chains.
        </li>
        <li>
            O problema apontado está acontecendo, porque a segunda chain está consumindo o callback da primeira chain.
        </li>
        <li>
            Como essa instância do callback já foi utilizada na readaptação da pergunta, ela fica incapaz de receber os tokens emitidos pela real resposta do LLM. No final das contas, recebemos como output a pergunta reformulada.
        </li>
    </ul>
 </div>

<h2 style='font-size:30px'> Isolating the Handler</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>   
            O LangChain nos permite passar callbacks como argumentos de diversas classes/operações.
        </li>
        <li>
            As ligações desse callback dependerão de onde ele for instanciado. Por exemplo, quando o informamos na instanciação de um LLM, ele será exclusivamente vinculado a esse.     
        </li>
        <li>
            No entanto, caso o passemos na invocação de uma chain - como é o caso de nosso projeto -, ele será ligado a todos os componentes que essa classe abrange. No caso de nossa StreamingConversationalRetrievalChain, tanto a chain de sintetização de textos, quanto a de resposta receberão o mesmo StreamingCallback, sendo por isso que elas o compartilham no uso da aplicação.
        </li>
    </ul>
 </div>

<h3 style='font-size:30px;font-style:italic'> Nossa Solução</h3>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Como não podemos informar o callback fora da invocação da chain - questões do projeto -, faremos uma breve customização do nosso programa. 
        </li>
        <li>
            Nas classes de LLM, definiremos o argumento `streaming` como False na chain de sintetização, e True na de output. Assim, instruiremos nosso callback a inserir dados na Queue, apenas caso `streaming=True`.
        </li>
    </ul>
 </div>

<h2 style='font-size:30px'> Streaming Complete!</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>   
            Para fazer o que propusemos, precisamos informar agora duas classes de LLM para nossa chain. Felizmente, a `ConversationalRetrievalChain` admite receber uma classe que será de uso exclusivo para a condensação das perguntas.
            <center style='margin-top:20px'> 
                <img src='img/11_condense_llm.png'>
            </center>
        </li>
    </ul>
 </div>

<p style='color:red'> Iniciei Aula 114; Aula 114 (2:00); Iniciar edição do callback</p>